# Intro
Klik bij iedere cell op 'Run'. Controleer de uitvoer. Notebooks geven een stap-voor-stap uitvoering van code, waardoor gekeken kan worden of het goed verloopt.

In [40]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

def to_xml(df, file_name=None, root_name='root', row_name='row', index_name='index', do_index=False, use_atts=False):
    root = ET.Element(root_name)
    tree = ET.ElementTree(root)
    for idx, row in df.iterrows():
        rowelm = ET.SubElement(root, row_name)
        if do_index:
            if use_atts:
                rowelm.set(index_name, str(idx))
            else:
                colelm = ET.SubElement(rowelm, index_name)
                colelm.text = str(idx)
        for hdr, val in row.iteritems():
            hdr = hdr.replace(' ', '_')
            if pd.notnull(val):
                val = str(val)
            else:
                val = ''
            if use_atts:
                rowelm.set(hdr, str(val))
            else:
                colelm = ET.SubElement(rowelm, hdr)
                colelm.text = str(val)
    if file_name is not None:
        tree.write(file_name, xml_declaration=False, encoding='utf8')
    return ET.tostring(root)

# Lees prerelease
De prerelease HTML tabel wordt van de LOINC site gelezen. Het kan even duren.

Controleer de output! Verwacht wordt een tabel zoals op de LOINC site te zien, als de LOINC HTML wijzigt kan dit niet goed gaan.

(Noot: de sumission queue heeft meer details, mogelijk kan dit ook een bron zijn.)

In [41]:
uri = 'https://loinc.org/prerelease/'
# uri = 'https://loinc.org/submissions/queue/'
prerelease = pd.read_html(uri)[0]
prerelease

,Created On,LOINC,Long Common Name,Special Use,Component,Prop,Time,System,Scale,Method,Shortname
0,2020-03-10,94531-1,SARS Coronavirus 2 RNA panel - Respiratory spe...,Special Use codes are developed in response to...,SARS coronavirus 2 RNA panel,-,Pt,Respiratory,-,Probe.amp.tar,SARS-CoV-2 RNA Pnl Resp NAA+probe
1,2020-03-10,94532-9,SARS coronavirus+SARS-like coronavirus+SARS co...,Special Use codes are developed in response to...,SARS coronavirus+SARS-like coronavirus+SARS co...,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS+SARS-Lk+SARS-CoV-2+MERS Ql NAA-prb
2,2020-03-10,94534-5,SARS coronavirus 2 RdRp gene [Presence] in Res...,Special Use codes are developed in response to...,SARS coronavirus 2 RdRp gene,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS-CoV-2 RdRp gene Resp Ql NAA+probe
3,2020-03-10,94533-7,SARS coronavirus 2 N gene [Presence] in Respir...,Special Use codes are developed in response to...,SARS coronavirus 2 N gene,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS-CoV-2 N gene Resp Ql NAA+probe
4,2020-03-09,94499-1,Respiratory viral pathogens DNA and RNA panel ...,Special Use codes are developed in response to...,Respiratory viral pathogens DNA & RNA panel,-,Pt,Respiratory,Ord,Probe.amp.tar,Resp viral path DNA+RNA Pnl Resp NAA+prb
...,...,...,...,...,...,...,...,...,...,...,...
293,2019-12-26,94147-6,Mitral valve B-bump by US.M-mode,NaN,B-bump,Find,Pt,Mitral valve,Ord,US.M-mode,MV B-bump M-mode
294,2019-12-26,94148-4,Descending thoracic aorta Holodiastolic flow r...,NaN,Blood flow reversal.holodiastolic,Find,Pt,Aorta.thoracic.descending,Ord,US.doppler,DTA Holodias flow reversal DOP
295,2019-12-17,94145-0,Crotonylglycine/Creatinine [Molar ratio] in Urine,NaN,Crotonylglycine/Creatinine,SRto,Pt,Urine,Qn,NaN,Crotonylglycine/Creat Ur-sRto
296,2019-12-17,94146-8,2-Decenedioate/Creatinine [Molar ratio] in Urine,NaN,2-Decenedioate/Creatinine,SRto,Pt,Urine,Qn,NaN,2-Decenedioate/Creat Ur-sRto


# Formatteren van de headers

In [42]:
prerelease.drop(['Created On', 'Special Use'], axis=1, inplace=True)
prerelease.columns = ['LOINC', 'longName', 'component', 'property', 'timing', 'system', 'scale', 'method', 'shortName']
prerelease.head()

,LOINC,longName,component,property,timing,system,scale,method,shortName
0,94531-1,SARS Coronavirus 2 RNA panel - Respiratory spe...,SARS coronavirus 2 RNA panel,-,Pt,Respiratory,-,Probe.amp.tar,SARS-CoV-2 RNA Pnl Resp NAA+probe
1,94532-9,SARS coronavirus+SARS-like coronavirus+SARS co...,SARS coronavirus+SARS-like coronavirus+SARS co...,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS+SARS-Lk+SARS-CoV-2+MERS Ql NAA-prb
2,94534-5,SARS coronavirus 2 RdRp gene [Presence] in Res...,SARS coronavirus 2 RdRp gene,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS-CoV-2 RdRp gene Resp Ql NAA+probe
3,94533-7,SARS coronavirus 2 N gene [Presence] in Respir...,SARS coronavirus 2 N gene,PrThr,Pt,Respiratory,Ord,Probe.amp.tar,SARS-CoV-2 N gene Resp Ql NAA+probe
4,94499-1,Respiratory viral pathogens DNA and RNA panel ...,Respiratory viral pathogens DNA & RNA panel,-,Pt,Respiratory,Ord,Probe.amp.tar,Resp viral path DNA+RNA Pnl Resp NAA+prb


# Maak XML
Het dataframe wordt geëxporteerd naar XML.
Vul target, usernaam, password in. Let op: eerst de cell runnen, dan waarden invullen. Cell niet opnieuw runnen, dat zet de default waarden terug.

In [43]:
xml = to_xml(prerelease, 'prerelease.xml', root_name='prerelease', row_name='concept')

In [44]:
import ipywidgets as widgets
targeturi = widgets.Dropdown(
    options=[('Productie', 'https://labterminologie.nl/lab/prerelease'),
             ('Acceptatie', 'https://terminologie.test-nictiz.nl/lab/prerelease'),
             ('Lokaal', 'http://localhost:8877/apps/lab-data/data/prerelease.xml')],
    description='Target:',
)
usr = widgets.Text(description='Username')
pwd = widgets.Password(description='Password')
widgets.VBox([
  targeturi, usr, pwd
])

In [45]:
targeturi.value

'http://localhost:8877/apps/lab-data/data/prerelease.xml'


# Upload naar de server
Verwachte uitvoer: (HTTP status) 201, dat is 'Created'

In [46]:
resp = requests.put(targeturi.value, data=xml, auth=requests.auth.HTTPBasicAuth(usr.value, pwd.value))
pwd.value = ''
resp.status_code

201

# Controleer de uitvoer
Onderstaande links voeren naar de prerelease tabel zoals zojuist geüpload.

[prerelease productie](http://labterminologie.nl/lab/prerelease)

[prerelease acceptatie](https://terminologie.test-nictiz.nl/lab/prerelease)

[prerelease lokaal](http://localhost:8877/apps/lab-data/data/prerelease.xml)